# Задание 1

**Устанавливаем** некоторые библиотеки

In [197]:
!pip install pymystem3
!pip install sklearn
!pip install bs4
!pip install nltk
!nltk.download('punkt')
!pip install fake_useragent

distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
"nltk.download" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\users\user\appdata\local\programs\python\python39\Include\UNKNOWN
sysconfig: c:\users\user\appdata\local\programs\python\python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


Затем **импортируем** библиотеки, необходимые для работы

In [179]:
from bs4 import BeautifulSoup
import string
punct_li = [word for word in string.punctuation]
import requests as req
import re
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')
from pymystem3 import Mystem
import time
from fake_useragent import UserAgent
import random
from collections import Counter
import copy
from sklearn.metrics import accuracy_score
m = Mystem()

Здесь я пытаюсь использовать юзерагент для того, чтобы понизить вероятность бана от сайта

In [180]:
ua = UserAgent(verify_ssl = False)
headers = {'User-Agent': ua.random}
count = {
    'number': 0
}
print(headers)

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2226.0 Safari/537.36'}


**Получаем сеты (множества)** по эмоциям

In [181]:
def getting_sets():
    set_positive = set()
    set_negative = set()
    counter_positive = Counter()
    counter_negative = Counter()
    banks = ['rsb', 'sovcombank']
    for bank in banks:
        count_positive = 0
        count_negative = 0
        for i in range(1, 25):
            if count_positive == 35 and count_negative == 35:
                break
            else:
                time.sleep(8)
                resp = req.get(f'https://www.banki.ru/services/responses/bank/{bank}/?is_countable=on&page={i}', 
                               proxies = {'https://135.181.61.25': '36135'}, headers = headers)
                soup = BeautifulSoup(resp.content, 'html.parser')
                stuff = soup.find_all('article', class_ = 'responses__item')
                for one in stuff:
                    if count_positive == 35 and count_negative == 35:
                        break
                    elif one.find('span', class_ = 'rating-grade rating-grade--small rating-grade--value-1') != None and count_negative < 35:
                        count_negative += 1
                        text = m.lemmatize(one.find('div', class_ = 'responses__item__message markup-inside-small markup-inside-small--bullet').getText().lower())
                        for word in text:
                            set_negative.add(word)
                            counter_negative[word] += 1
                        print(str('count_negative: ' + str(count_negative)))
                    elif one.find('span', class_ = 'rating-grade rating-grade--small rating-grade--value-5') != None and count_positive < 35:
                        count_positive += 1
                        text = m.lemmatize(one.find('div', class_ = 'responses__item__message markup-inside-small markup-inside-small--bullet').getText().lower())
                        for word in text:
                            set_positive.add(word)
                            counter_positive[word] += 1
                        print(str('count_positive: ' + str(count_positive)))
                    else:
                        continue
    return [set_positive, set_negative, counter_positive, counter_negative]

**Очищаем сеты** от различных символов, которые могут помешать в оценке

In [182]:
def cleaning(list_of_sets):
    set_positive = copy.copy(list_of_sets[0])
    for word in list_of_sets[0]:
        if list_of_sets[2][word] < 2 or word in ru_stopwords or re.search(' |\n|\t|.\xa0|[0-9]', word) != None or word in punct_li or word == '':
            set_positive.discard(word)
    set_negative = copy.copy(list_of_sets[1])
    for word in list_of_sets[1]:
        if list_of_sets[3][word] < 2 or word in ru_stopwords or re.search(' |\n|\t|.\xa0|[0-9]', word) != None or word in punct_li or word == '':
            set_negative.discard(word)
    return [set_positive, set_negative, list_of_sets[2], list_of_sets[3]]

Ниже я **проверяю работу** этих функций

In [183]:
sets = cleaning(getting_sets())

count_negative: 1
count_negative: 2
count_negative: 3
count_negative: 4
count_positive: 1
count_positive: 2
count_positive: 3
count_negative: 5
count_positive: 4
count_negative: 6
count_negative: 7
count_positive: 5
count_negative: 8
count_positive: 6
count_positive: 7
count_positive: 8
count_positive: 9
count_negative: 9
count_positive: 10
count_positive: 11
count_negative: 10
count_positive: 12
count_positive: 13
count_negative: 11
count_positive: 14
count_negative: 12
count_positive: 15
count_negative: 13
count_positive: 16
count_positive: 17
count_positive: 18
count_positive: 19
count_positive: 20
count_positive: 21
count_positive: 22
count_positive: 23
count_positive: 24
count_negative: 14
count_negative: 15
count_positive: 25
count_positive: 26
count_negative: 16
count_positive: 27
count_negative: 17
count_positive: 28
count_positive: 29
count_positive: 30
count_positive: 31
count_negative: 18
count_positive: 32
count_positive: 33
count_negative: 19
count_positive: 34
count_posit

In [184]:
positive_start = {elem for elem in sets[0]}
sets[0].difference_update(sets[1])
sets[1].difference_update(positive_start)

In [185]:
print(sets[0])

{'рост', 'гл', 'рамка', ']–', 'ко', 'внимательно', 'набирать', 'успех', 'клиентоориентированный', 'случаться', 'ольга', 'надбавка', 'нарушать', 'профессионализм', 'выбор', 'приятный', 'положение', 'посещать', 'жкх', 'грамотно', 'смотря', 'качественно', 'доброжелательный', 'ирина', 'месячный', 'полтора', 'подробность', 'рад', 'четко', 'официальный', 'входить', 'современный', 'мария', 'добровольный', 'код', 'додо', 'консультация', 'довольный', 'преимущество', 'место', 'расстраиваться', 'понятный', 'анна', 'сергей', 'клиентоориентированность', 'равный', 'ч', 'дистанционно', 'заключаться', 'юлия', 'родственник', 'любить', 'повышение', 'процветание', 'всякий', 'отзывчивость', 'чадо', 'б', '")', 'понимание', 'прошлый', 'обрадовать', 'банковский', 'род', 'целое', 'александр', 'вид', 'руководство', 'активация', 'особенно', 'режим', 'добропорядочный', 'деталь', 'samsung', 'поведение', 'огромный', 'светлана', 'называть', 'отзывчивый', 'встречать', 'вовремя', 'милый', 'подарок', 'удовлетворять', 

In [186]:
print(sets[1])

{'online', 'ожидание', 'спрашивать', 'обходить', 'уведомлять', 'целый', 'согласный', 'тишина', 'пенсия', 'почему', 'идентичный', 'заказывать', 'скидка', 'вынуждать', 'странный', 'связываться', 'поверять', 'машина', 'лист', 'цифра', 'контактный', 'недвижимость', 'привлекательный', 'неизвестный', 'обновляться', 'распоряжение', 'отписка', 'разглашаться', 'немного', 'завтра', 'среда', 'выход', 'видимо', 'недовольный', 'мама', 'видеть', 'гражданин', 'имущество', 'консультант', 'мера', 'оформляться', 'запрашивать', 'пытаться', 'обзвон', 'свидетельство', 'прописка', 'центральный', 'озвучивать', 'тип', 'добро', 'заявлять', 'шок', 'отказывать', 'максимально', '?!', 'обязанность', 'наглый', 'пересчет', 'ходить', 'что-то', 'необходимо', 'бесплатный', 'добиваться', 'заполнять', 'контакт', 'возмещение', 'скорый', 'продавец', 'навязнуть', 'сбп', 'требовать', 'мясо', 'практически', 'тело', 'дублировать', 'прекращаться', 'фото', 'причем', 'сталкиваться', 'тинькофф', 'обманывать', 'основание', 'бот', '

Все отлично работает. Теперь **выносим вердикт по отзывам** с помощью сетов

In [187]:
def checking(review, sets):
    positive_words = int()
    negative_words = int()
    lemmatized = m.lemmatize(review.lower())
    for word in lemmatized:
        if word in sets[0]:
            positive_words += 1
        elif word in sets[1]:
            negative_words += 1
    if positive_words > negative_words:
        return 'positive'
    elif positive_words < negative_words:
        return 'negative'
    else:
        return 'positive' #сеты несбалансированы, поэтому я решил, что при равном количестве слов преимущество 
                          #надо отдавать эмоции с меньшим количеством слов

Ниже показаны проверочные отзывы. Первый является негативным, второй позитивным

In [188]:
review = '''Действительно, бизнес "Русского Стандарта" строится на надувательстве заемщиков! Это самое настоящее "кидалово" для клиентов Банка. Бешенные проценты, завуалированные под низкопроцентные кредиты. Нулевые кредиты Банк выдает только под неликвидные товары, по сговору с фирмой-продавцом. Причем на такие товары делается большая наценка, которая в последствии делится между Банком и торговой организацией. Товары продаются в магазинах-партнерах Банка отвратительного качества (3-4 сорт), очень много бракованных и возвращаются. покупателями в течении 1-2 месяцев после покупки. Брак по низким ценам из Эльдорадо и Евросети обычное дело.
В отношении своих сотрудников Банк занимается подставами и кидаловым.
Так что не надо брать "дешевые" кредиты в этом Банке, и вообще не имейте с ним дело и другим скажите.
Проще обратиться в другие Банки, чтобы не чувствовать себя потом в дураках!'''

In [189]:
review2 = '''Не знаю как кредиты в этом банке, но вот ДЕПОЗИТЫ просто класс! Мало того, что высокие ставки, бесплатно выпускают депозитную карту, на которую тоже начисляются проценты  А когда мне срочно понадобились мои деньги, я смогла закрыть свой вклад, позвонив по телефону и досрочно закрыв депозит. Уже на следующий день получила деньги с моей карты!!!'''

In [190]:
checking(review, sets)

'negative'

In [191]:
checking(review2, sets)

'negative'

Как видим, программа работает не всегда. Тогда **посмотрим ее точность** на примере 10 тестовых отзывов

In [192]:
def getting_tests():
    reviews_emotions = dict()
    count_positive = 0
    count_negative = 0
    for i in range(60, 85):
        time.sleep(8)
        resp = req.get(f'https://www.banki.ru/services/responses/bank/rsb/?is_countable=on&page={i}', 
                       proxies = {'https://135.181.61.25': '36135'}, headers = headers)
        soup = BeautifulSoup(resp.content, 'html.parser')
        stuff = soup.find_all('article', class_ = 'responses__item')
        for one in stuff:
            if count_positive == 15 and count_negative == 15:
                return reviews_emotions
            elif one.find('span', class_ = 'rating-grade rating-grade--small rating-grade--value-1') != None and count_negative < 15:
                text = one.find('div', class_ = 'responses__item__message markup-inside-small markup-inside-small--bullet').getText()
                count_negative += 1
                reviews_emotions[text] = 'negative'
                print(str('count_negative: ' + str(count_negative)))
            elif one.find('span', class_ = 'rating-grade rating-grade--small rating-grade--value-5') != None and count_positive < 15:
                text = one.find('div', class_ = 'responses__item__message markup-inside-small markup-inside-small--bullet').getText()
                count_positive += 1
                reviews_emotions[text] = 'positive'
                print(str('count_positive: ' + str(count_positive)))
            else:
                continue

In [193]:
test_reviews = getting_tests()

count_negative: 1
count_negative: 2
count_positive: 1
count_positive: 2
count_positive: 3
count_negative: 3
count_negative: 4
count_negative: 5
count_negative: 6
count_negative: 7
count_positive: 4
count_negative: 8
count_positive: 5
count_negative: 9
count_negative: 10
count_negative: 11
count_negative: 12
count_negative: 13
count_negative: 14
count_negative: 15
count_positive: 6
count_positive: 7
count_positive: 8
count_positive: 9
count_positive: 10
count_positive: 11
count_positive: 12
count_positive: 13
count_positive: 14
count_positive: 15


In [194]:
def calculating_accuracy(reviews_emotions, sets):
    random_ten = list(reviews_emotions.items())
    random.shuffle(random_ten)
    random_ten = random_ten[:10]
    results = list()
    reality = list()
    for review in random_ten:
        reality.append(review[1])
        result = checking(review[0], sets)
        results.append(result)
    print(results)
    print(reality)
    print('Accuracy: ' + str(accuracy_score(results, reality)))

In [195]:
calculating_accuracy(test_reviews, sets)

['negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive']
['positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive']
Accuracy: 0.8


### Какие улучшения могут быть имплементированы?
Во-первых, следует сбалансировать сеты по эмоциям, так как иначе одна из эмоций может иметь преимущество при оценке отзывов. Во-вторых, можно улучшить процесс оценки с помощью каунтеров, чтобы учитывать частотность тех или иных слов. В-третьих, можно банально увеличить выборку, взяв большее количество банков и отзывов. 